### Приложение для чтения книг

### Шаг 1. Загрузка данных
- загрузить данные.

### Шаг 2. Ответы на задания проекта
- посчитать, сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее количество книг толще 50 страниц;
- определить автора с самой высокой средней оценкой книг;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Шаг 3. Общий вывод
- написать общий вывод.

### Шаг 1. Загрузка данных


In [1]:
# импортириуем необходимые библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# получим доступ к данным
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# первые пять строк таблицы books
query_books = ''' SELECT *
        FROM books
        LIMIT 5
    '''
books = pd.io.sql.read_sql(query_books, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# первые пять строк таблицы authors
query_authors = ''' SELECT *
        FROM authors
        LIMIT 5
    '''
authors = pd.io.sql.read_sql(query_authors, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# первые пять строк таблицы ratings
query_ratings = ''' SELECT *
        FROM ratings
        LIMIT 5
    '''
ratings = pd.io.sql.read_sql(query_ratings, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# первые пять строк таблицы reviews
query_reviews = ''' SELECT *
        FROM reviews
        LIMIT 5
    '''
reviews = pd.io.sql.read_sql(query_reviews, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
# первые пять строк таблицы publishers
query_publishers = ''' SELECT *
        FROM publishers
        LIMIT 5
    '''
publishers = pd.io.sql.read_sql(query_publishers, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Вывод
- По первичному визуальному анализу с данными все хорошо. Содержимое колонок не перепутано местами и в целом данные выглядят аккуратно. 

### Шаг 2. Ответы на задания проекта

In [8]:
# посчитаем сколько книг было выпущено после 1 января 2000 года
books_count = '''SELECT 
COUNT (book_id) AS cnt
FROM books
WHERE publication_date > '2000-01-01'
'''
books_count = pd.io.sql.read_sql(books_count, con = engine)
books_count

,cnt
0,819


 - После 1 января 2000 года было выпущено 819 книг. 

In [9]:
# посчитаем для каждой книги количество обзоров и среднюю оценку
reviews_ratings = '''SELECT
        ratings.book_id AS book_id,
        COUNT(reviews.text) AS total_reviews,
        AVG(ratings.rating) AS mean_rating
        FROM
            ratings
        LEFT JOIN reviews ON 
            (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
        GROUP BY 
            ratings.book_id
        ORDER BY 
            ratings.book_id
'''
reviews_ratings = pd.io.sql.read_sql(reviews_ratings, con = engine)
reviews_ratings

,book_id,total_reviews,mean_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


 - Для каждой книги посчитано количество отзывов и средний рейтинг.

In [10]:
#посмотрим, у какого издательства больше всего книг с толщиной свыше 50 страниц
publishers_books = '''SELECT
        publishers.publisher AS publisher,
        COUNT(books.book_id) AS total_books
        FROM
            books
        LEFT JOIN publishers ON 
            publishers.publisher_id = books.publisher_id
        GROUP BY 
            publishers.publisher
        HAVING
            SUM(books.num_pages) > 50
        ORDER BY 
            total_books DESC
        LIMIT 5
'''
publishers_books = pd.io.sql.read_sql(publishers_books, con = engine)
publishers_books

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


- Больше всего книг с толщиной свыше 50 страниц выпустило издательство Penguin Books - 42 книги;
- На втором месте Vintage - 31 книга;
- На третьем Grand Central Publishing - 25 книг;
- На четвертом Penguin Classics - 24 книги;
- На пятом Bantam - 19 книг. 

In [13]:
#найдем автора с самой высокой средней оценкой книг среди книг, получивших свыше 50 отзывов
author_rate = '''SELECT
    authors.author,
    AVG(filtered.average_rating) AS avg_rating
    FROM
        (SELECT
            book_id AS book_id,
            COUNT(rating_id) AS rating_counts,
            AVG(rating) AS average_rating
        FROM
            ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50) 
        AS filtered
    INNER JOIN books ON books.book_id = filtered.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY authors.author
    ORDER BY avg_rating DESC
    LIMIT 10
'''
author_rate = pd.io.sql.read_sql(author_rate, con = engine)
author_rate

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


- Джоан Роулинг является автором с самой высокой средней оценкой - 4.28. Впрочем, это неудивительно.

In [12]:
# посчитаем среднее количество обзоров от пользователей, оставивших более 50 отзывов
mean_reviews = '''
        SELECT
        AVG(sub.count)
        FROM
        (SELECT COUNT(*)
            FROM reviews
            WHERE username IN
                (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50)
            GROUP BY username)
            AS sub;
        '''

reviews = pd.io.sql.read_sql(mean_reviews, con = engine)
reviews

,avg
0,24.333333


- В среднем пользователи, поставившие более 50 оценок, оставляет 24 рецензии. 

### Шаг 3. Общий вывод
- Если ориентироваться на данные, то после 1 января 2000 года вышло 819 книг. Интересно, мне кажется, что выпускается гораздо больше книг. 
- Издательство Penguin Books является лидером выпущенному количеству книг с более чем 50 страницами. Это неудивительно, так как оно является одним из самых популярных издательств на зарубежном книжном рынке. К тому же, если учитывать разнообразные его ответвления, например. Penguin Classics, то получится и того больше. 
- Джоан Роулинг является самым высокооценным автором из списка. Вполне ожидаемый результат, учитывая ее аудиторию.
- Пользователи, поставившие более 50 оценок, в среднем оставляют 24 рецензии. Как правило, то чем больше люди читает и сильнее вовлечены в условный goodreads или livelib, то тем чаще они оставляют рецензии, тем более, что в последнее время сервисы стимулируют пользователей оставлять рецензии. 